In [1]:
import sys 
sys.path.append("../../../../rllm/dataloader")

import time
import argparse
import numpy as np

# import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score

# from utils import load_data

from load_data import load_data
from models import GraphSage
from utils import adj_matrix_to_list, multihop_sampling

t_total = time.time()

In [2]:
args_cuda = True


np.random.seed(42)
torch.manual_seed(42)
if args_cuda:
    torch.cuda.manual_seed(42)

In [3]:
# Load data
data, adj, features, labels, idx_train, idx_val, idx_test = load_data('movielens-regression')
# labels = labels.argmax(dim=-1)

node_index = torch.arange(0, adj.shape[0])
node_index_user = torch.arange(0, adj.indices()[0, -1]+1)
node_index_movie = torch.arange(adj.indices()[1, 0], adj.indices()[1, -1]+1)
node_index_train_user = torch.unique(adj.indices()[:, idx_train][0],return_counts=False)
node_index_train_movie = torch.unique(adj.indices()[:, idx_train][1],return_counts=False)
node_index_test_user = torch.unique(adj.indices()[:, idx_test][0],return_counts=False)
node_index_test_movie = torch.unique(adj.indices()[:, idx_test][1],return_counts=False)

label_mat = torch.sparse_coo_tensor(adj.indices(), labels, size=adj.shape, requires_grad=False).float()
label_mat_test = torch.sparse_coo_tensor(adj.indices()[:, idx_test], labels[idx_test], size=adj.shape, requires_grad=False).float()
test_adjacency_dict, test_label_dict = adj_matrix_to_list(label_mat_test, node_index_test_movie, label_mat_test)
adjacency_dict, label_dict = adj_matrix_to_list(adj, node_index_movie, label_mat)

In [12]:
args_hidden = [64]
args_lr = 0.01
args_weight_decay = 5e-4
args_epochs = 400
NUM_NEIGHBORS_LIST = [35]
NUM_BATCH_PER_EPOCH = 10
batch_size = 128
model = GraphSage(input_dim=features.shape[1], hidden_dim=args_hidden,
                  num_neighbors_list=NUM_NEIGHBORS_LIST)
optimizer = optim.Adam(model.parameters(),
                       lr=args_lr, weight_decay=args_weight_decay)
loss_func = nn.MSELoss()
# loss_func = F.cross_entropy
DEVICE = "cpu"

if args_cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    node_index = node_index.cuda()
    node_index_movie = node_index_movie.cuda()
    node_index_user = node_index_user.cuda()
    node_index_train_user = node_index_train_user.cuda()
    node_index_train_movie = node_index_train_movie.cuda()
    node_index_test_user = node_index_test_user.cuda()
    node_index_test_movie = node_index_test_movie.cuda()


def train(epoch):
    t = time.time()
    model.train()
    loss_lst = []
    if epoch % 50 == 0: test()
    for batch in range(NUM_BATCH_PER_EPOCH):
        optimizer.zero_grad()
        # sample from node_index_movie first
        batch_node_movie_train = node_index_movie[torch.randint(0, len(node_index_movie), (batch_size,))]
        batch_sampling_result, batch_sampling_label = multihop_sampling(batch_node_movie_train, NUM_NEIGHBORS_LIST, adjacency_dict, label_dict)
        batch_sampling_x = [features[idx].float().to(DEVICE) for idx in batch_sampling_result]
        output = model(batch_sampling_x)
        batch_sampling_label = F.one_hot(batch_sampling_label.long()-1, 5).float().to(DEVICE)
        loss_train = loss_func(output, batch_sampling_label)
        loss_lst.append(loss_train.detach().item())
        loss_train.backward()
        optimizer.step()


    # print('Epoch: {:04d}'.format(epoch+1),
    #         'loss_train: {:.4f}'.format(sum(loss_lst)/len(loss_lst)),
    #         'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    with torch.no_grad():
        batch_sampling_result, batch_sampling_label = multihop_sampling(node_index_test_movie, NUM_NEIGHBORS_LIST, adjacency_dict, label_dict)
        batch_sampling_x = [features[idx].float().to(DEVICE) for idx in batch_sampling_result]
        pred = model.test(batch_sampling_x, node_index_test_movie, test_adjacency_dict, features)
        pred_list = []
        label_list = []
        for key, value in pred.items():
            pred_list.extend(value.tolist())
            label_list.extend(test_label_dict[key].tolist())
        pred_list = torch.tensor(pred_list)
        label_list = torch.tensor(label_list)
        print("test MSE:", F.mse_loss(pred_list, label_list))


# test()
# Train model
t_total = time.time()
for epoch in range(args_epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

test MSE: tensor(3.0228)
test MSE: tensor(1.6363)
test MSE: tensor(1.7731)
test MSE: tensor(1.6132)
test MSE: tensor(1.5880)
test MSE: tensor(1.6271)
test MSE: tensor(1.6864)
test MSE: tensor(1.6684)
Optimization Finished!
Total time elapsed: 43.1524s
test MSE: tensor(1.6901)
